# Analysis

This notebook contains routines for analyzing the output of keypoint-MoSeq.

```{note}
To use the (optional) interactive widgets below, you must launch jupyter from a terminal in which the keypoint_moseq environment is active. 
```

## Setup

We assume you have already have keypoint-MoSeq outputs that are organized as follows.
```
<project_dir>/               ** current working directory
└── <model_name>/            ** model directory
    ├── results.h5           ** model results
    └── grid_movies/         ** [Optional] grid movies folder
```
Use the code below to define your project directory and model name, and to check if the required jupyter extensions are enabled. If the jupyter extensions are not enabled and you would like to use the optional interactive widgets below, try running the [installation commands](https://keypoint-moseq.readthedocs.io/en/latest/install.html#install-jupyter-extensions) and confirm that jupyter was launched from the `keypoint_moseq` environment, as mentioned in the note above. 

In [ ]:
import keypoint_moseq as kpms

project_dir='path/to/project' # the full path to the project directory
model_name='model_name' # name of model to analyze (e.g. something like `2023_05_23-15_19_03`)

## Assign Groups

The goal of this step is to assign group labels (such as "mutant" or "wildtype") to each recording. These labels are important later for performing group-wise comparisons. The annotations are saved to a to  `{project_dir}/index.csv`.

**Option 1: Interactive group setting widget**

Launch `interactive_group_setting` using the code below.

- Click cells in the "group" column to edit the group name.
- When you are done adding group labels, use `Save group info` to save current group assignments.
  
If the widget does not appear, try restarting your Jupyter Notebook kernel and re-run the cells. Alternatively, you can edit index.csv in Excel or LibreOffice Calc.

In [ ]:
kpms.interactive_group_setting(project_dir, model_name)

## Generate dataframes

Generate a pandas dataframe called `moseq_df` that contains syllable labels and kinematic information for each frame across all the recording sessions. 

In [ ]:
moseq_df = kpms.compute_moseq_df(project_dir, model_name, smooth_heading=True) 
moseq_df

In [ ]:
import os
save_dir = os.path.join(project_dir, model_name) # directory to save the moseq_df dataframe
moseq_df.to_csv(os.path.join(save_dir, 'moseq_df.csv'), index=False)
print('Saved `moseq_df` dataframe to', save_dir)

Next generate a dataframe called `stats_df` that contains summary statistics for each syllable in each recording session, such as its usage frequency and its distribution of kinematic parameters.

In [ ]:
stats_df = kpms.compute_stats_df(
    project_dir,
    model_name,
    moseq_df, 
    min_frequency=0.005,       # threshold frequency for including a syllable in the dataframe
    groupby=['group', 'name'], # column(s) to group the dataframe by
    fps=30)                    # frame rate of the video from which keypoints were inferred

stats_df

In [ ]:
import os
save_dir = os.path.join(project_dir, model_name)
stats_df.to_csv(os.path.join(save_dir, 'stats_df'), index=False)
print('Saved `stats_df` dataframe to', save_dir)

##  Label syllables

**Interactive widget**

The following cell invoke an interactive widget to assign a name and short description to each syllable for downstream interpretation. These annotations are saved to `{project_dir}/{model_name}/syll_info.csv`.

- Select the syllable to from the dropdown to display syllable grid movie.
- Click `label` in the target syllable row and enter a syllable name.
- [Optional] Click `short description` in the target syllable row and enter a short decription.
- When you are done adding syllable labels, use `Save syllable info` to save current group assignments.

If the widget does not appear, try restarting your Jupyter Notebook kernel and re-run the cells. Alternatively, you can edit syll_info.csv in Excel or LibreOffice Calc.

In [ ]:
kpms.label_syllables(project_dir, model_name, moseq_df) 

## Compare between groups

Test for statistically significant differences between groups of recordings. The code below takes a syllable property (e.g. frequency or duration), plots its disribution for each syllable across for each group, and also tests whether the property differs significantly between groups. The results are summarized in a plot that is saved to `{project_dir}/{model_name}/analysis_figures`.

There are two options for setting the order of syllables along the x-axis. When `order='stat'`, syllables are sorted by the mean value of the statistic. When `order='diff'`, syllables are sorted by the magnitude of difference between two groups that are determined by the `ctrl_group` and `exp_group` keywords. Note `ctrl_group` and `exp_group` are not related to significance testing.

In [ ]:
kpms.plot_syll_stats_with_sem(
    stats_df, project_dir, model_name,
    plot_sig=True,    # whether to mark statistical significance with a star
    thresh=0.05,      # significance threshold
    stat='frequency', # statistic to be plotted (e.g. 'duration' or 'velocity_px_s_mean')
    order='stat',     # order syllables by overall frequency ("stat") or degree of difference ("diff")
    ctrl_group='a',   # name of the control group for statistical testing
    exp_group='b',    # name of the experimental group for statistical testing
    figsize=(8, 4),   # figure size    
    groups=stats_df['group'].unique(), # groups to be plotted
);

### Transition matrices
Generate heatmaps showing the transition frequencies between syllables.

In [ ]:
normalize='bigram' # normalization method ("bigram", "rows" or "columns")

trans_mats, usages, groups, syll_include=kpms.generate_transition_matrices(
    project_dir, model_name, normalize=normalize,
    min_frequency=0.005 # minimum syllable frequency to include
)    

kpms.visualize_transition_bigram(
    project_dir, model_name, groups, trans_mats, syll_include, normalize=normalize, 
    show_syllable_names=True # label syllables by index (False) or index and name (True)
)

### Syllable Transition Graph
Render transition rates in graph form, where nodes represent syllables and edges represent transitions between syllables, with edge showing transition rate for each pair of syllables (secifically the max of the two transition rates in each direction). The code below first generates a transition graph for each single group, and then generates a difference-graph for each pair of groups.

In [ ]:
kpms.plot_transition_graph_group(
    project_dir, model_name, 
    groups, trans_mats, usages, syll_include, 
    layout='circular',        # transition graph layout ("circular" or "spring")
    show_syllable_names=False # label syllables by index (False) or index and name (True)
)

In [ ]:
kpms.plot_transition_graph_difference(project_dir, model_name, 
                                      groups, trans_mats, usages, syll_include, 
                                      layout='circular') # transition graph layout ("circular" or "spring")